In [1]:
import os
from PIL import Image
from helpers.image_preprocessing import frame_transform
from torchvision import models
import torch
import lmdb
import json
import glob

In [2]:
root_dir = 'D:/frames-resampled'

def get_all_dirs():
    frames_dirs = os.listdir(root_dir)
    frames_dirs.sort()
    return frames_dirs

def update_progress(prog):
    with open('frame-feature-progress.json', 'w') as file:
        json.dump(prog, file)

def get_progress():
    if not os.path.exists('frame-feature-progress.json'):
        return []
    with open('frame-feature-progress.json', 'r') as file:
        prog = json.load(file)
    return prog

In [3]:
feature_dir = 'D:/frame-features'
map_size = 512 * 1024 * 1024
env = lmdb.open(feature_dir, map_size=map_size)

def write_to_lmdb(data_dict):
    print('writing to lmdb')
    with env.begin(write=True) as txn:
        for key, value in data_dict.items():
            txn.put(str(key).encode(), value.tobytes())

In [4]:
all_dirs = get_all_dirs()
progress = get_progress()
remain_dirs = [frame_dir for frame_dir in all_dirs if frame_dir not in progress]

In [5]:
feature_dict = {}
resnet = models.resnet34()
feature_extractor = torch.nn.Sequential(*list(resnet.children())[:-1]).to('cuda')
feature_extractor.eval()

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Con

In [6]:
for i, frame_dir in enumerate(remain_dirs):
    print(f'Processing {frame_dir} ({i+1}/{len(remain_dirs)})')
    frame_paths = glob.glob(os.path.join(root_dir, frame_dir, '*.jpg'))

    tensor_list = []
    for frame_path in frame_paths:
        frame = Image.open(frame_path).convert('RGB')
        tensor_list.append(frame_transform(frame).unsqueeze(0))

    frame_tensor = torch.cat(tensor_list, dim=0).to('cuda')
    with torch.no_grad():
        features = feature_extractor(frame_tensor)
        pooled_features = torch.mean(features, dim=0).squeeze().cpu().numpy()

    feature_dict[frame_dir] = pooled_features
    progress.append(frame_dir)

    if (i+1) % 100 == 0 or (i+1) == len(remain_dirs):
        write_to_lmdb(feature_dict)
        update_progress(progress)
        feature_dict = {}

Processing 1305440 (1/61108)
Processing 1305450 (2/61108)
Processing 1305470 (3/61108)
Processing 1305490 (4/61108)
Processing 1305500 (5/61108)
Processing 1305540 (6/61108)
Processing 1305570 (7/61108)
Processing 1305590 (8/61108)
Processing 1305610 (9/61108)
Processing 1305640 (10/61108)
Processing 1305660 (11/61108)
Processing 1305690 (12/61108)
Processing 1305720 (13/61108)
Processing 1305730 (14/61108)
Processing 1305760 (15/61108)
Processing 1305780 (16/61108)
Processing 1305820 (17/61108)
Processing 1305880 (18/61108)
Processing 1305890 (19/61108)
Processing 1305940 (20/61108)
Processing 1305960 (21/61108)
Processing 1305970 (22/61108)
Processing 1305990 (23/61108)
Processing 1306010 (24/61108)
Processing 1306020 (25/61108)
Processing 1306060 (26/61108)
Processing 1306110 (27/61108)
Processing 1306180 (28/61108)
Processing 1306210 (29/61108)
Processing 1306240 (30/61108)
Processing 1306570 (31/61108)
Processing 1306580 (32/61108)
Processing 1306620 (33/61108)
Processing 1306630 

In [1]:
import lmdb

env = lmdb.open('D:/frame-features', readonly=True)

with env.begin() as txn:
    s = txn.stat()
    print(s)

env.close()


{'psize': 4096, 'depth': 3, 'branch_pages': 3, 'leaf_pages': 438, 'overflow_pages': 69908, 'entries': 69908}
